In [ ]:
recording_path = "output.log.1"

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pylab import *

import plotly
print plotly.__version__            # version 1.9.x required
plotly.offline.init_notebook_mode() # run at the start of every notebook

In [ ]:
data = pd.read_csv(
    recording_path,
    header=None,
    names=['datetime','msg','lbl_str','str','lbl_esc','esc','lbl_aa','ax','ay','az','lbl_angle','heading','lbl_rpm','rpm_raw','rpm','rpm_delta','rpm_odo','lbl_ping','ping','lbl_odo','odo','lbl_ms','ms','lbl_us','us','lbl_ypr','yaw','pitch','roll'],
    dtype=str,
    index_col=0,
    parse_dates = True,
    error_bad_lines = False)

In [ ]:
(data.index[4] - data.index[3]).total_seconds()


In [ ]:
data = data[(data['msg'] == 'TRACE_DYNAMICS')]
for c in data.columns:
  data = data[(data[c] != None)]
h = data.head()

In [ ]:
def float_or_nan(a):
  try:
    return float(a)
  except:
    return float('nan')

In [ ]:
import math
def force_float(df,column):
    df[column] = df[column].apply(float_or_nan)
    bad = df[column].apply(math.isnan)
    df = df[bad==False]
    return df

In [ ]:
for column in ['str','esc','ax','ay','az','heading','rpm_raw','rpm','rpm_delta','rpm_odo','ping','odo','ms','us','yaw','pitch','roll']:
    print 'cleansing', column
    print 'rows before', len(data)
    data = force_float(data,column)
    print 'rows after', len(data)

In [ ]:
data['secs'] = (data.index - data.index[0]) / np.timedelta64(1, 's')
#data['secs'] 
data['odo_ticks_per_sec'] = data.odo.diff() / data.secs.diff()
data['meters_per_sec'] = data['odo_ticks_per_sec'] * 0.003411
data['mph'] = data['meters_per_sec'] * 2.23694
data['odo_ax'] = data.meters_per_sec.diff(30)/data.secs.diff(30)
#data.speed = data.odo.diff()/data.secs.diff()

In [ ]:
#remove when we are standing still
data = data[(data['mph']!=0)]
len(data)

In [ ]:
hist(data.esc,bins=20)



ft = data.query('esc > 1900 & esc <= 1910')

In [ ]:
#plot(ft.mph, ft.ax)
plotly.offline.iplot({
    "data": [
            {"x": ft.mph, "y":ft.ax, "mode":"markers"},
            ],
    "layout": {
        "title": 'acceleration vs speed in 1900-1910 range'
    }
})

In [ ]:
#ticklabel_format(style='plain')

plot(data.secs,data.odo_ticks_per_sec)
plot(data.secs,data.rpm)

t = data.index-data.index[0]


In [ ]:
plot(data.odo_ticks_per_sec)

In [ ]:

plotly.offline.iplot({
    "data": [
            {"x": data.secs,"y": data.rpm,"name":'rpm'}, 
            {"x": data.secs, "y":data.odo_ticks_per_sec, "name":'odo_ticks_per_sec'},
            {"x": data.secs, "y":data.heading, "name":'heading'}],
    "layout": {
        "title": recording_path
    }
})

In [ ]:
plotly.offline.iplot({
    "data": [
            {"x": data.secs, "y":data.heading, "name":'heading'}],
    "layout": {
        "title": recording_path
    }
})

In [ ]:
plotly.offline.iplot({
    "data": [
            {"x": data.secs, "y":data.ax, "name":'ax'},
            {"x": data.secs, "y":data.ay, "name":'ay'},
            {"x": data.secs, "y":data.az, "name":'az'}],
    "layout": {
        "title": recording_path
    }
})

In [ ]:
plotly.offline.iplot({
    "data": [
            {"x": data.secs, "y":data.az/4, "name":'az (scaled)'},
            #{"x": data.secs, "y":data.rpm*48./32., "name":'rpm*48/32'},
            {"x": data.secs, "y":data.odo_ticks_per_sec, "name":'odo_ticks_per_sec'}],
    "layout": {
        "title": recording_path
    }
})

In [ ]:
plotly.offline.iplot({
    "data": [
            {"x": data.secs, "y":data.meters_per_sec, "name":'meters_per_sec'}
            ],
    "layout": {
        "title": recording_path
    }
})

In [ ]:
plotly.offline.iplot({
    "data": [
            {"x": data.secs, "y":data.mph, "name":'mph',"mode":"markers"},
            {"x": data.secs, "y":(data.esc-1500)/20., "name":'esc',"mode":'markers'},
            {"x": data.secs, "y":data.ax, "name":'ax', "mode":'markers'}
            ],
    "layout": {
        "title": recording_path
    }
})

In [ ]:
plotly.offline.iplot({
    "data": [
            {"x": data.secs, "y":data.esc, "name":'esc'}
            ],
    "layout": {
        "title": recording_path
    }
})

In [ ]:
plotly.offline.iplot({
    "data": [
            {"x": data.secs, "y":data.ax, "name":'ax'},
            {"x": data.secs, "y":data.ay, "name":'ay'},
            {"x": data.secs, "y":data.ay, "name":'az'},
            ],
    "layout": {
        "title": recording_path
    }
})

In [ ]:
plotly.offline.iplot({
    "data": [
            {"x": data.secs,"y": data.meters_per_sec,"name":'m/s'}, 
            {"x": data.secs, "y":data.ax, "name":'ax'},
            {"x": data.secs, "y":data.odo_ax, "name":'odo_ax'}
            ],
    "layout": {
        "title": recording_path
    }
})